In [424]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import maximum_filter
from matplotlib.colors import LogNorm
from astropy.io import fits
from PIL import Image
from astropy.visualization import astropy_mpl_style
from pprint import pprint


In [425]:
def cargar_imagenes_fits(numero_imagenes, formato):
    """
    Descripcion: Carga las imágenes FITS y en una lista.
    Input:
        - lista_imagenes: Número total de imágenes FITS a cargar.
        - ('{:03d}.fit'.format(i)) se refiere a un archivo con 3 ceros y el ultimo digito representado por "i"
            - Si i es 1, el resultado de '{:03d}.fit'.format(i) será '001.fit'.
            - IMPORTANTE: Segun el set de imagenes los nombres varian de la forma "00i.fit" y "000i.fit", por lo que las entradas disponibles son {:03d} y {:04d}.
    Output:
        - lista_imagenes: Lista que contiene las imágenes FITS cargadas.
    """
    lista_imagenes = []
    for i in range(numero_imagenes):
        fits_file = fits.open((formato + '.fit').format(i))
        imagen = fits_file[0].data
        lista_imagenes.append(imagen)
    print("Carga",i,"lista!")
    return lista_imagenes

In [426]:
def image_subtract(imagedata,imagedata2):
    """
    Descripcion: funcion para restar 2 imagenes formato .fit
        Input:
            - imagedata, imagedata2: Data de la imagen en formato "fit", extraida de la forma "Imagen_fit.[0].data".
        Output: 
            - subtracted_img_rotated: Variable que guarda la resta de la data de ambas imagenes. 
    """
    #Variables
    angulo_rotacion = 0    # La imagen puede ser rotada si se cambia el valor del angulo de rotacion (k= N*90°. k=0 -> no rotacion)

    # Resta en valor absoluto  
    subtracted_img = cv2.absdiff(imagedata, imagedata2)

    # Rotacion de imagen (k= N*90°. k=0 -> no rotacion)
    subtracted_img_rotated = np.rot90(subtracted_img, angulo_rotacion)
    return (subtracted_img_rotated)


In [427]:
def brillo_contraste(imagen_png):
    """
    Descripción: Esta función ajusta los valores de brillo y contraste de una imagen en formato PNG y la guarda en un archivo llamado 'convertida_b_c.png'. 
        donde:
            - imagen_modificada = alpha * imagen_original + beta
            - alpha: Factor de contraste que controla el contraste de la imagen. Valores mayores que 1 aumentan el contraste, mientras que valores menores que 1 lo disminuyen.
            - beta: Factor de brillo agregado a cada píxel de la imagen. Valores positivos aumentan el brillo, mientras que valores negativos lo disminuyen.
    Input:
        - imagen_png: Imagen en formato PNG que se va a ajustar.
    Output: 
        - 'convertida_b_c.png': Imagen resultante después de ajustar el brillo y el contraste.
    """
    #Variables
    alpha = 1.9                             # alpha: Factor de contraste.
    beta = 60                               # beta: Factor de brillo agregado.
    nombre_salida = 'convertida_b_c.png'    # Nombre del archivo de salida, (cambiarlo podria afectar el funcionamiento del programa).
    
    resultado = cv2.convertScaleAbs(imagen_png, alpha=alpha, beta=beta)
    cv2.imwrite(nombre_salida, resultado)

In [428]:
def fit_png(imagen_png):
    """
    Descripcion: toma la imagen en formato PNG, como una matriz NumPy, y ajusta el rango de valores para que esté en un formato similar al de orignen de 16 bits (2^16 = 65536). 
        Esto se logra restando el valor mínimo de todos los píxeles de la imagen y luego multiplicando por el factor que escala los valores al rango de 0 a 65535. 
        Esto asegura que la imagen tenga una representación adecuada para ser almacenada como un archivo PNG de la misma calidad.
        
        Input:
            - imagedata: Data de la imagen en formato "fit", extraida de la forma "Imagen_fit.[0].data".
        Output: 
            - fit_convertido.png: Mismo archivo de imagen guardado en fotmato .png   
    """  
    # Convertir los datos a un array NumPy y ajustar el rango de valores a 16 bit (2^16 = 65536)
    imagen_png = (imagen_png - imagen_png.min()) * (65535.0 / (imagen_png.max() - imagen_png.min()))
    imagen_png = imagen_png.astype('uint16')

    # Crear una imagen PIL a partir del array NumPy
    imagen = Image.fromarray(imagen_png)

    # Guardar la imagen como PNG
    imagen.save('fit_convertido.png') 
    return imagen




In [429]:
def contornos(imagen_png, index):
    """
    Descripción: Esta función detecta los dos contornos más grandes en una imagen y los dibuja en color verde sobre una copia de la imagen original. 
        - Luego guarda la imagen resultante con los contornos dibujados.
        - IMPORTANTE: La siguiente funcion (sumar_imagenes()) trabaja con los contornos en color verde solamente.

    Input:
        - imagen_png: Imagen en formato PNG.
        - index (i): Índice utilizado para generar un nombre de archivo único para la imagen de salida.
    
    Output:
        - 'resta(index-9).png': Imagen resultante con los dos contornos más grandes dibujados en color verde.

    """
    # Variables
    area_porcentaje = 0.2                              # Porcentaje del contorno mayor que deben tener como minimo los otros contornos
    grosor_linea = 2                                   # Ajusta el grosor de la linea (en pixeles) que dibuja los contornos
    umbral_pixeles = 190                               # Cualquier píxel en la imagen que tenga un valor menor será convertido a negro (0)
    pixel_blanco = 255                                 # Valor que se asigna a los píxeles que superan el umbral (se convierten en blanco, 8 bits = 255)
    nombre_salida = 'resta({}-9).png'.format(index)    # Nombre del archivo de salida, (cambiarlo podria afectar el funcionamiento del programa).


    # Convierte la imagen a escala de grises y binariza la imagen utilizando un umbral adecuado para detectar el segmento blanco
    gray = cv2.cvtColor(imagen_png, cv2.COLOR_BGR2GRAY)
    _, binaria = cv2.threshold(gray, umbral_pixeles, pixel_blanco, cv2.THRESH_BINARY)

    # Buscar contornos en la imagen binarizada y los organiza de mayor a menor área
    contornos, _ = cv2.findContours(binaria, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contornos = sorted(contornos, key=cv2.contourArea, reverse=True)

    # Si se detectan al menos dos contornos
    if len(contornos) >= 1:

        # Encontrar el primer contorno más grande
        mayor_contorno = contornos[0]

        # Inicializar una lista de contornos seleccionados, incluyendo el contorno más grande
        contornos_seleccionados = [mayor_contorno]

        # Calcular el área del mayor contorno
        area_mayor_contorno = cv2.contourArea(mayor_contorno)

        # Agregar contornos similares al contorno más grande
        for contorno in contornos:  
            
            # Calcular el área del contorno actual
            area_contorno_actual = cv2.contourArea(contorno)
            # Calcular el área relativa
            area_relativa = area_contorno_actual / area_mayor_contorno
            
            # Si el área relativa es mayor al 30% y no es el contorno más grande, agregar el contorno actual al conjunto de contornos final
            if area_relativa > area_porcentaje and not np.array_equal(contorno, mayor_contorno):
                contornos_seleccionados.append(contorno)

        # Si hay al menos un contorno seleccionado
        if len(contornos_seleccionados) >= 1:

            # Toma el contorno más grande y lo dibuja en una copia de la imagen original para luego guardarla como archivo
            contorno_mayor1 = contornos_seleccionados[0]
            imagen_con_contornos = imagen_png.copy()
            # Dibuja el contorno de color verde = ((0, 255, 0)), con un grosor determinado
            cv2.drawContours(imagen_con_contornos, [contorno_mayor1], -1, (0, 255, 0), grosor_linea)                

            # Si hay al menos dos contornos seleccionados
            if len(contornos_seleccionados) > 1:

                # Toma todos los contornos y los dibuja en una copia de la imagen original para luego guardarla como archivo
                contorno_mayor2 = contornos_seleccionados[1]
                imagen_con_contornos = imagen_png.copy()
                # Dibuja el contorno de color verde = ((0, 255, 0)), con un grosor determinado
                cv2.drawContours(imagen_con_contornos, [contorno_mayor1], -1, (0, 255, 0), grosor_linea)
                cv2.drawContours(imagen_con_contornos, [contorno_mayor2], -1, (0, 255, 0), grosor_linea)
                cv2.imwrite(nombre_salida, imagen_con_contornos)
        else:
            print("No se encontraron contornos.")
            return



In [430]:
def sumar_imagenes(nombres_archivos):
    """
    Descripcion: Esta función suma los contornos marcados de color verde de una lista de imágenes para formar una sola y esta la convierte a escala de grises.
        Input:
            - nombres_archivos: Lista de nombres de archivos de las imágenes a sumar.
        Output: 
            - img_sumada.png: Imagen resultante de la suma de las partes verdes de todas las imágenes.
    """
    #Variables
    umbral_verde = 200                  # Los píxeles con un valor mayor o igual a este umbral se considerarán verdes.
    valor_maximo = 255                  # valor máximo que pueden tomar los píxeles verdes en la máscara binaria resultante.
    nombre_salida = 'img_sumada.png'    # Nombre del archivo de salida, (cambiarlo podria afectar el funcionamiento del programa).


    # Verificar que haya al menos una imagen para sumar
    if len(nombres_archivos) < 1:
        print('\nNo hay suficientes imágenes para sumar.\n')
        return

    # Leer la primera imagen como fondo y la convierte a escala de grises
    fondo = cv2.imread(nombres_archivos[0])
    fondo_gris = cv2.cvtColor(fondo, cv2.COLOR_BGR2GRAY)

    # Recorrer todas las demás imágenes y agregar solo las partes verdes al fondo
    for nombre_archivo in nombres_archivos[1:]:
        # Leer la imagen
        imagen = cv2.imread(nombre_archivo)
        
        # Convertir la imagen a escala de grises
        imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

        # Separar el canal verde de la imagen actual
        canal_verde = imagen[:, :, 1]

        # Encontrar los píxeles verdes en la imagen actual utilizando un umbral
        _, green_mask = cv2.threshold(canal_verde, umbral_verde, valor_maximo, cv2.THRESH_BINARY)

        # Agregar los píxeles verdes al fondo
        fondo_gris[green_mask == valor_maximo] = valor_maximo

    # Se guardar el resultado de la suma
    cv2.imwrite(nombre_salida, fondo_gris)
    print('\nSuma de todas las imágenes realizada y guardada como {}\n'.format(nombre_salida))

In [431]:
def recta():
    """
    Descripción: Esta función encuentra el contorno más grande en una imagen binarizada y ajusta una línea recta que atraviesa este contorno utilizando RANSAC. Luego, dibuja la línea recta en una copia de la imagen original y guarda las imágenes resultantes.

    Input: No recibe ningún parámetro de entrada directo, pero asume que la imagen 'img_final.png' está presente en el directorio donde se ejecuta la función.

    Output:
        - 'contorno_mas_grande.png': Imagen resultante con el contorno más grande dibujado en verde.
        - 'imagen_final.png': Imagen resultante con la línea recta proyectada sobre la imagen original.
        
    """
    # Variables
    area_porcentaje = 0.3                              # Porcentaje del contorno mayor que deben tener como minimo los otros contornos
    umbral_pixeles = 190                               # Cualquier píxel en la imagen que tenga un valor menor será convertido a negro (0)
    pixel_blanco = 255                               # Valor que se asigna a los píxeles que superan el umbral (se convierten en blanco, 8 bits = 255)

    # Cargar la imagen original en escala de grises y se binariza utilizando umbrales adecuados para detectar los sehmentos blancos
    img_final = cv2.imread('img_sumada.png')
    gray = cv2.cvtColor(img_final, cv2.COLOR_BGR2GRAY)
    _, binaria = cv2.threshold(gray, umbral_pixeles, pixel_blanco, cv2.THRESH_BINARY)

    # Buscar contornos en la imagen binarizada y guardamos el del area mayor
    contornos, _ = cv2.findContours(binaria, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mayor_contorno = max(contornos, key=cv2.contourArea)
    area_mayor_contorno = cv2.contourArea(mayor_contorno)

    # Inicializar el contorno final con el contorno más grande, y el segundo como una lista vacia
    contorno_final = mayor_contorno.copy()
    contornos_seleccionados = []

    # Agregar contornos similares al contorno más grande
    for contorno in contornos:  
        
        # Calcular el área del contorno actual y el area relativa (relativa al mayor contorno)
        area_contorno_actual = cv2.contourArea(contorno)
        area_relativa = area_contorno_actual / area_mayor_contorno
        
        # Si el área relativa es mayor a cierto porcentaje y no es el contorno más grande, agregar el contorno actual al conjunto de contornos final
        if area_relativa > area_porcentaje and not np.array_equal(contorno, mayor_contorno):
            contorno_final = np.concatenate((contorno_final, contorno), axis=0)  
            contornos_seleccionados.append(contorno)

    # Guarda y dibuja el contorno más grande y todos los contornos seleccionados en una nueva imagen c/u (de color verde = (0, 255, 0))
    contorno_mas_grande = np.zeros_like(img_final)                                         #Se crea una matriz de ceros del mismo tamaño y tipo que la imagen img_final. 
    contornos_img = np.zeros_like(img_final)                                               #Esta matriz se utilizará para dibujar el contorno más grande.
    cv2.drawContours(contorno_mas_grande, [contorno_final], -1, (0, 255, 0), cv2.FILLED)   # Se dibuja el contorno en la matriz (de color verde (0, 255, 0)). 
    cv2.drawContours(contornos_img, contornos_seleccionados, -1, (0, 255, 0), cv2.FILLED)  # y se rellena completamente (cv2.FILLED).
    cv2.imwrite('contorno_mas_grande.png', contorno_mas_grande)
    cv2.imwrite('contornos_secundarios.png', contornos_img)

    # Encontrar los puntos extremos del contorno final
    extremos_izquierdos = tuple(contorno_final[contorno_final[:, :, 0].argmin()][0])
    extremos_derechos = tuple(contorno_final[contorno_final[:, :, 0].argmax()][0])
    extremos_superiores = tuple(contorno_final[contorno_final[:, :, 1].argmin()][0])
    extremos_inferiores = tuple(contorno_final[contorno_final[:, :, 1].argmax()][0])

    # Obtener los coeficientes de la línea a partir de los puntos extremos encontrados
    x1 = extremos_inferiores[0]
    y1 = extremos_inferiores[1]
    x2 = extremos_superiores[0]
    y2 = extremos_superiores[1]
    x3 = extremos_izquierdos[0]
    y3 = extremos_izquierdos[1]
    x4 = extremos_derechos[0]
    y4 = extremos_derechos[1]

    # Calcular los puntos medios entre los puntos inferiores y superiores de ambas rectas
    x_promedio_3 = (x3 + x2) / 2
    y_promedio_3 = (y3 + y2) / 2
    x_promedio_4 = (x1 + x4) / 2
    y_promedio_4 = (y1 + y4) / 2

    # Dibujar las líneas rectas en "imagen_sumada.png"
    recta_extendida = img_final.copy()
    height, width = img_final.shape[:2]


    # Extendemos las rectas, si la línea no es vertical (es decir, x2 != x1 o x4 != x3 o x_promedio_4 != x_promedio_3), se calcula la pendiente de la línea.
    # Primera recta (en verde (0, 255, 0))
    if x2 != x1:
        slope_1 = (y2 - y1) / (x2 - x1)
        y1_extended = int(y1 - slope_1 * x1)
        y2_extended = int(slope_1 * (width - 1) + y1 - slope_1 * x1)
        cv2.line(recta_extendida, (0, y1_extended), (width - 1, y2_extended), (0, 255, 0), 1)

    # Segunda recta (en rojo (255, 0, 0))
    if x4 != x3:
        slope_2 = (y4 - y3) / (x4 - x3)
        y3_extended = int(y3 - slope_2 * x3)
        y4_extended = int(slope_2 * (width - 1) + y3 - slope_2 * x3)
        cv2.line(recta_extendida, (0, y3_extended), (width - 1, y4_extended), (255, 0, 0), 1)

    # Tercera recta (promedio primera y segunda) (en morado (255, 0, 255))
    if x_promedio_4 != x_promedio_3:
        slope_avg = (y_promedio_4 - y_promedio_3) / (x_promedio_4 - x_promedio_3)
        y_avg_3_extended = int(y_promedio_3 - slope_avg * x_promedio_3)
        y_avg_4_extended = int(slope_avg * (width - 1) + y_promedio_3 - slope_avg * x_promedio_3)
        cv2.line(recta_extendida, (0, y_avg_3_extended), (width - 1, y_avg_4_extended), (255, 0, 255), 2)


    # Dibujar círculos alrededor de los puntos superiores e inferiores de las rectas 
    cv2.circle(recta_extendida, (x1, y1), 2, (0, 0, 0), -1)  # Punto inferior de la primera recta
    cv2.circle(recta_extendida, (x2, y2), 2, (0, 0, 0), -1)  # Punto superior de la primera recta
    cv2.circle(recta_extendida, (x3, y3), 2, (0, 0, 0), -1)  # Punto inferior de la segunda recta
    cv2.circle(recta_extendida, (x4, y4), 2, (0, 0, 0), -1)  # Punto superior de la segunda recta


    # Guardar la imagen_rectas con todas las rectas proyectadas (opcional)
    
    cv2.imwrite('imagen_rectas.png', recta_extendida)
    print("Imagen 'imagen_rectas.png' guardada correctamente.")
    
    # Crear una nueva imagen solo con la tercera recta promedio extendida en la imagen original
    img_con_recta = img_final.copy()
    cv2.line(img_con_recta, (0, y_avg_3_extended), (width - 1, y_avg_4_extended), (255, 0, 255), 2)
    cv2.imwrite('img_final.png', img_con_recta)
    print("Imagen 'img_final.png' guardada correctamente.")

    # Imprimir las coordenadas de los puntos extremos y las pendientes de ambas rectas
    print("\nCoordenadas de Recta 1: \nPunto inferior: ", extremos_inferiores, "   Punto superior: ", extremos_superiores)
    print("Pendiente de la primera recta:", slope_1)

    print("\nCoordenadas de Recta 2:  \nPunto inferior", (-x2, -y2),"Punto superior: " , (-x1, -y1))
    print("Pendiente de la segunda recta:", slope_2)

    print("\nCoordenadas de Recta promedio: \nPunto inferior: ", (x_promedio_3, y_promedio_3),"   Punto superior: ", (x_promedio_4, y_promedio_4))
    print("Pendiente de la tercera recta (promedio):", slope_avg)


In [432]:
def limpiar_archivos(activado):
    """
    Descripción: Esta función limpia los archivos utilizados si la limpieza está activada.

    Input:
        - activado: Booleano que indica si se debe realizar la limpieza de archivos.
    """
    archivos_utilizados = ["fit_convertido.png", "convertida_b_c.png", "contorno_mas_grande.png", "contornos_secundarios.png", "img_sumada.png"]
    
    if activado:
        for archivo in archivos_utilizados:
            if os.path.exists(archivo):
                os.remove(archivo)
                print(f"\nArchivo '{archivo}' eliminado.")
            else:
                print(f"\nEl archivo '{archivo}' no existe.")

In [433]:
def procesar_imagenes_fits(lista_imagenes, formato):
    """
    Descripción: Esta función procesa una lista de imágenes FITS aplicando una serie de operaciones, incluyendo resta de imágenes, ajuste de brillo y contraste, detección de contornos y suma de partes específicas de las imágenes. Además, utiliza la función 'recta' para encontrar el contorno más grande en una imagen binarizada y ajustar una línea recta que lo atraviesa utilizando el algoritmo RANSAC.

    Input:
        - lista_imagenes: Lista que contiene las imágenes FITS a procesar.

    Output:
        La función produce varias imágenes resultantes que son guardadas en el directorio de trabajo:
        - 'resta(index-9).png': Imágenes resultantes de la resta de cada imagen con la imagen de referencia '009.fit'.
        - 'img_sumada.png': Imagen resultante de la suma de las partes verdes de todas las imágenes.
        - 'contorno_mas_grande.png': Imagen resultante con el contorno más grande dibujado en verde.
        - 'imagen_final.png': Imagen resultante con la línea recta proyectada sobre la imagen original.
    """
    nombres_archivos = []
    for i, imagen in enumerate(lista_imagenes):
            # Ultima imagen del set como referencia ({:04N}9.fit), con N como el nuemero de digitos
            fits_file_ref = fits.open((formato.format(i)[:-1] + '9.fit'))
            
            imagen_ref = fits_file_ref[0].data

            # Restar la imagen actual con la imagen de referencia
            imagen_resta = image_subtract(imagen, imagen_ref)
            fit_png(imagen_resta)
            img_convertida = cv2.imread("fit_convertido.png")
            
            # Aplicar brillo y contraste a la imagen resultante de la resta
            brillo_contraste(img_convertida)
            imagen = cv2.imread("convertida_b_c.png")

            # Detectar contornos en la imagen procesada y obtener los nombres de archivo generados
            contornos(imagen, index=i)
            nombres_archivos.append('resta({}-9).png'.format(i))
    sumar_imagenes(nombres_archivos)
    recta()


### ***Funciones y variables importantes***

### **Variables principales:**
    - imagedata1, imagedata2: Data de las imágenes en formato "fit", extraída de la forma "Imagen_fit.[0].data".
    - imagen_png: Imagen en formato "png".

### **Archivos:**
    - imagen.fit: Archivo que contiene los datos de la imagen en formato "fit".
    - resultado_diferencia.fit**: Imagen resultante de la resta de las datas de ambas imágenes.
    - fit_convertido.png: Conversión de "resultado_diferencia.fit" a formato ".png" para modificar valores de brillo y contraste.
    - convertida_b_c.png: Imagen resultado de la modificación de brillo y contraste de "fit_convertido.png".
    - resta(index-9).png: Imágenes resultantes de la función canny_img(), donde index es el índice utilizado para generar un nombre de archivo único.

### ***Resumen de pasos del programa***

El programa sigue los siguientes pasos:

1. **Resta de imágenes**:
    - Se realizan operaciones de resta entre dos imágenes en formato ".fit".
    - Se rota la imagen resultante según un ángulo especificado.

2. **Conversión de formato**:
    - Se convierte la imagen resultante de formato ".fit" a ".png".

3. **Ajuste de brillo y contraste**:
    - Se ajustan los valores de brillo y contraste de la imagen en formato ".png".

4. **Detección de contornos**:
    - Se detectan los dos contornos más grandes en una imagen en formato ".png".
    - Los contornos detectados se dibujan en una copia de la imagen original en verde y se guarda la imagen resultante.

5. **Suma de imágenes**:
    - Se suman partes específicas de varias imágenes ".png" en una sola imagen.
    - Se asume que las partes a sumar son las áreas verdes en las imágenes.

6. **Ajuste de línea recta**:
    - Se encuentra el contorno más grande en una imagen binarizada.
    - Se ajusta una línea recta que atraviesa este contorno.
    - Se dibuja la línea recta en una copia de la imagen original y se guarda la imagen resultante.


### ***Descripción de funciones***

- **image_subtract(imagedata1, imagedata2)**:
    - Esta función resta dos imágenes en formato ".fit" y devuelve la imagen resultante con la opción de rotarla.

- **brillo_contraste(imagen_png)**:
    - Ajusta los valores de brillo y contraste de una imagen en formato ".png" y la muestra en una ventana que se puede cerrar con cualquier tecla.

- **fit_png(imagen_png)**:
    - Convierte una imagen en formato ".fit" a ".png".

- **canny_img(imagen_png, index)**:
    - Detecta los dos contornos más grandes en una imagen y los dibuja sobre una copia de la imagen original en verde. Luego guarda la imagen resultante con los contornos dibujados.

- **sumar_imagenes(nombres_archivos)**:
    - Suma partes específicas de varias imágenes ".png" en una sola imagen, asumiendo que las partes a sumar son las áreas verdes.

- **recta()**:
    - Encuentra el contorno más grande en una imagen binarizada y ajusta una línea recta que atraviesa este contorno. Luego dibuja la línea recta en una copia de la imagen original y guarda las imágenes resultantes.





In [434]:
""" PROGRAMA PRINCIPAL """

# Definir el formato como una variable
formato = '{:04d}'

# Cargar las imágenes FITS en una lista
lista_imagenes = cargar_imagenes_fits(9, formato)

# Procesar las imágenes
procesar_imagenes_fits(lista_imagenes, formato)

# Activar o desactivar la limpieza de archivos
limpiar_archivos(activado = False)  # Cambiar a False si no se desea limpiar los archivos

Carga 8 lista!

Suma de todas las imágenes realizada y guardada como img_sumada.png

Imagen 'imagen_rectas.png' guardada correctamente.
Imagen 'img_final.png' guardada correctamente.

Coordenadas de Recta 1: 
Punto inferior:  (2723, 2299)    Punto superior:  (2832, 2166)
Pendiente de la primera recta: -1.2201834862385321

Coordenadas de Recta 2:  
Punto inferior (-2832, -2166) Punto superior:  (-2723, -2299)
Pendiente de la segunda recta: -0.9669421487603306

Coordenadas de Recta promedio: 
Punto inferior:  (2774.5, 2226.0)    Punto superior:  (2780.5, 2234.0)
Pendiente de la tercera recta (promedio): 1.3333333333333333
